# 🏡 California Housing Price Prediction

## 📦 1. Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 📂 2. Load & Prepare Data

In [ ]:
data = fetch_california_housing()
X, y = data.data, data.target

print(X.shape, y.shape)  # (20640, 8) (20640,)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## 🔥 3. PyTorch Model

In [ ]:
# Convert to tensors
x_train_t = torch.tensor(x_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
x_test_t = torch.tensor(x_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(x_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define model
model_torch = nn.Sequential(
    nn.Linear(8, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(128, 64), nn.BatchNorm1d(64), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(64, 32), nn.BatchNorm1d(32), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(32, 16), nn.BatchNorm1d(16), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(16, 8), nn.BatchNorm1d(8), nn.ReLU(),
    nn.Linear(8, 1)
)

criterion = nn.MSELoss()
optimizer = optim.Adam(model_torch.parameters(), lr=0.001)

# Training loop
epochs = 100
train_losses, val_losses = [], []
for epoch in range(epochs):
    model_torch.train()
    epoch_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model_torch(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    train_losses.append(epoch_loss / len(train_loader))

    model_torch.eval()
    with torch.no_grad():
        val_outputs = model_torch(x_test_t)
        val_loss = criterion(val_outputs, y_test_t).item()
        val_losses.append(val_loss)

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_loss:.4f}")

## 📈 4. PyTorch Evaluation

In [ ]:
model_torch.eval()
with torch.no_grad():
    y_pred_t = model_torch(x_test_t).numpy()
    y_true_t = y_test_t.numpy()

mse = mean_squared_error(y_true_t, y_pred_t)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_t, y_pred_t)
r2 = r2_score(y_true_t, y_pred_t)

print(f"Final Results (PyTorch): MSE={mse:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, R2={r2:.4f}")

plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs"); plt.ylabel("MSE Loss")
plt.title("PyTorch Training vs Validation Loss")
plt.legend(); plt.show()

plt.scatter(y_true_t, y_pred_t, alpha=0.5)
plt.xlabel("True Values"); plt.ylabel("Predicted Values")
plt.title("PyTorch: Predicted vs True")
plt.plot([y_true_t.min(), y_true_t.max()], [y_true_t.min(), y_true_t.max()], 'r--')
plt.show()

## 🔵 5. Keras Sequential Model

In [ ]:
model_keras = Sequential([
    Dense(64, input_dim=8, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1)
])
model_keras.compile(optimizer='adam', loss='mse', metrics=['mse'])
model_keras.summary()

early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
history = model_keras.fit(
    x_train, y_train,
    epochs=300, batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

## 📊 6. Keras Sequential Evaluation

In [ ]:
y_pred_k = model_keras.predict(x_test)
mse = mean_squared_error(y_test, y_pred_k)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_k)
r2 = r2_score(y_test, y_pred_k)

print(f"Final Results (Keras Sequential): MSE={mse:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, R2={r2:.4f}")

plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.xlabel("Epochs"); plt.ylabel("MSE Loss")
plt.title("Keras Training vs Validation Loss")
plt.legend(); plt.show()

plt.scatter(y_test, y_pred_k, alpha=0.5)
plt.xlabel("True Values"); plt.ylabel("Predicted Values")
plt.title("Keras Sequential: Predicted vs True")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()

## 🟣 7. Keras Functional API Model

In [ ]:
inputs = Input(shape=(X.shape[1],))
x = Dense(128)(inputs)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(4, activation='relu')(x)
outputs = Dense(1)(x)

model_func = Model(inputs, outputs)
model_func.compile(optimizer='adam', loss='mse', metrics=['mse'])
model_func.summary()

early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
history_func = model_func.fit(
    x_train, y_train,
    epochs=300, batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

## 🏁 8. Conclusion
- ✅ Compared PyTorch, Keras Sequential, and Keras Functional API models.  
- 📌 Reported metrics: **MSE, RMSE, MAE, R²**.  
- 🚀 Future improvements: hyperparameter tuning, deeper networks, experimenting with dropout rates.